In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt

# Feature Engineering

In [29]:
df = pd.read_csv("/Users/eva/Documents/Study/Y1S2/DMT/assignment2/cleaned_training_set_VU_DM.csv")
df.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,bool_visitor_hist
0,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,893,3,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0
1,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,10404,4,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0
2,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,21315,3,4.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0
3,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,27348,2,4.0,...,0.0,0.0,0.0,-1.0,0.0,5.0,0,0.0,0,0
4,1,2013-04-04 08:32:15,12,187,0.0,0.0,219,29604,4,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0


In [30]:
df_test = pd.read_csv("/Users/eva/Documents/Study/Y1S2/DMT/assignment2/cleaned_test_set_VU_DM.csv")

In [31]:
# print columns with NA in df and df_test 
print(df.columns[df.isnull().any()].tolist())
print(df_test.columns[df_test.isnull().any()].tolist())


[]
[]


In [32]:
#print df and df_test length
print(len(df))
print(len(df_test))

4958347
4959183


## Find columns with numeric values so that we can caluculate mean, std, median per prop_id


In [33]:
combined_df = pd.concat([df, df_test], ignore_index=False)

In [34]:
bool_flag_cols = [col for col in combined_df.columns if 'bool' in col.lower() or 'flag' in col.lower()]

# transform the columns to boolean
for col in bool_flag_cols:
    combined_df[col] = combined_df[col].astype(bool)


In [35]:
print(combined_df['date_time'].dtypes)
# transform it into datetime
combined_df['date_time'] = pd.to_datetime(combined_df['date_time'])


object


In [36]:
# print columns end with rate and env
rate_cols = combined_df.columns[combined_df.columns.str.endswith('rate')]
inv_cols = [col for col in combined_df.columns if 'inv' in col.lower()]
print(rate_cols)
print(inv_cols)

Index(['comp1_rate', 'comp2_rate', 'comp3_rate', 'comp4_rate', 'comp5_rate',
       'comp6_rate', 'comp7_rate', 'comp8_rate'],
      dtype='object')
['comp1_inv', 'comp2_inv', 'comp3_inv', 'comp4_inv', 'comp5_inv', 'comp6_inv', 'comp7_inv', 'comp8_inv']


In [37]:
numeric_cols = [col for col in combined_df.select_dtypes(include=['int64', 'float64']).columns if 'id' not in col and col not in rate_cols and col not in inv_cols]
print(numeric_cols)
print(len(numeric_cols))

['visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_starrating', 'prop_review_score', 'prop_location_score1', 'prop_location_score2', 'prop_log_historical_price', 'position', 'price_usd', 'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_query_affinity_score', 'orig_destination_distance', 'comp1_rate_percent_diff', 'comp2_rate_percent_diff', 'comp3_rate_percent_diff', 'comp4_rate_percent_diff', 'comp5_rate_percent_diff', 'comp6_rate_percent_diff', 'comp7_rate_percent_diff', 'comp8_rate_percent_diff', 'gross_bookings_usd']
25


In [38]:
features = ['visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_starrating', 'prop_review_score', 'prop_location_score1', 
            'prop_location_score2', 'prop_log_historical_price', 'price_usd', 'srch_length_of_stay', 'srch_booking_window', 
            'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_query_affinity_score', 'orig_destination_distance', 
            'comp1_rate_percent_diff', 'comp2_rate_percent_diff']


# 计算每个组合的均值、中位数和标准差
grouped = combined_df.groupby(['prop_id', 'srch_id', 'srch_destination_id'])
stats = grouped[features].agg(['mean', 'median', 'std'])

original_train_len = len(df)
original_test_len = len(df_test)
for feature in features:
    for id_col in ['prop_id', 'srch_id', 'srch_destination_id']:
        for stat in ['mean', 'median', 'std']:
            column_name = f'{feature}_{id_col}_{stat}'
            combined_df[column_name] = grouped[feature].transform(stat)

/var/folders/st/hfszkgw55n73h3v2jz7n8xth0000gn/T/ipykernel_2878/2445899447.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df[column_name] = grouped[feature].transform(stat)
/var/folders/st/hfszkgw55n73h3v2jz7n8xth0000gn/T/ipykernel_2878/2445899447.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df[column_name] = grouped[feature].transform(stat)
/var/folders/st/hfszkgw55n73h3v2jz7n8xth0000gn/T/ipykernel_2878/2445899447.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the res

In [39]:
# add a column to compute the price difference in each srch_id group(compared with the median one)
combined_df['price_diff'] = combined_df['price_usd'] - combined_df['price_usd_srch_id_median']

# add a column to compute prop_location_score1 difference in each srch_id group(compared with the median one)
combined_df['prop_location_score1_diff'] = combined_df['prop_location_score1'] - combined_df['prop_location_score1_srch_id_median']

# add a column to compute prop_location_score2 difference in each srch_id group(compared with the median one)
combined_df['prop_location_score2_diff'] = combined_df['prop_location_score2'] - combined_df['prop_location_score2_srch_id_median']

# add a column to compute review score difference in each srch_id group(compared with the median one)
combined_df['prop_review_score_diff'] = combined_df['prop_review_score'] - combined_df['prop_review_score_srch_id_median']

# add a column to compute prop_starrating_diff in each srch_id group(compared with the median one)
combined_df['prop_starrating_diff'] = combined_df['prop_starrating'] - combined_df['prop_starrating_srch_id_median']

# add columns to compute price per adult and price per person
combined_df['price_per_adult'] = combined_df['price_usd'] / combined_df['srch_adults_count']
combined_df['price_per_person'] = combined_df['price_usd'] / (combined_df['srch_adults_count'] + combined_df['srch_children_count'])

# add prop_historical_price
combined_df['prop_historical_price'] = np.exp(combined_df['prop_log_historical_price'])

# add prop_clicked_prob by taking 10 to the power of “srch query affinity score”
combined_df['prop_clicked_prob'] = 10 ** combined_df['srch_query_affinity_score']

# add child or not bool
combined_df['child_bool'] = combined_df['srch_children_count'] > 0



/var/folders/st/hfszkgw55n73h3v2jz7n8xth0000gn/T/ipykernel_2878/1980084526.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df['price_diff'] = combined_df['price_usd'] - combined_df['price_usd_srch_id_median']
/var/folders/st/hfszkgw55n73h3v2jz7n8xth0000gn/T/ipykernel_2878/1980084526.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df['prop_location_score1_diff'] = combined_df['prop_location_score1'] - combined_df['prop_location_score1_srch_id_median']
/var/folders/st/hfszkgw55n73h3v2jz7n8xth0000gn/T/ipyk

In [40]:
df = combined_df.iloc[:original_train_len]
df_test = combined_df.iloc[original_train_len:original_train_len + original_test_len]

In [41]:
na_std_cols = [col for col in df.columns if 'std' in col and df[col].isna().any()]
df.loc[:, na_std_cols] = df.loc[:, na_std_cols].fillna(0)

In [42]:
# print columns with NA in df
print(df.columns[df.isnull().any()].tolist())

[]


In [43]:
na_std_cols = [col for col in df_test.columns if 'std' in col and df_test[col].isna().any()]
df_test.loc[:, na_std_cols] = df_test.loc[:, na_std_cols].fillna(0)

In [44]:
# print columns with NA in df_test
print(df_test.columns[df_test.isnull().any()].tolist())

['position', 'gross_bookings_usd']


In [45]:
# delete columns with NA in df_test
df_test = df_test.dropna(axis=1)

In [46]:
#to_csv
df.to_csv('/Users/eva/Documents/Study/Y1S2/DMT/assignment2/feature_engineered_training_set_VU_DM.csv', index=False)

In [47]:
df_test.to_csv('/Users/eva/Documents/Study/Y1S2/DMT/assignment2/feature_engineered_test_set_VU_DM.csv', index=False)

In [48]:
# print df_test length
print(len(df_test))
print(len(df))

4959183
4958347


In [49]:
# truncate into chunks
# Load the large CSV file
file_path = '/Users/eva/Documents/Study/Y1S2/DMT/assignment2/feature_engineered_training_set_VU_DM.csv'
# Define the size of each chunk
chunk_size = 500000  # This number can change 

# Split the CSV into chunks
for i in range(0, len(df), chunk_size):
    chunk = df.iloc[i:i + chunk_size]
    chunk.to_csv(f'/Users/eva/Documents/Study/Y1S2/DMT/assignment2/feature_engineered_training_chunk_{i//chunk_size}.csv', index=False)

In [50]:
# truncate into chunks
# Load the large CSV file
file_path = '/Users/eva/Documents/Study/Y1S2/DMT/assignment2/feature_engineered_training_set_VU_DM.csv'
# Define the size of each chunk
chunk_size = 500000  # This number can change 

# Split the CSV into chunks
for i in range(0, len(df_test), chunk_size):
    chunk = df_test.iloc[i:i + chunk_size]
    chunk.to_csv(f'/Users/eva/Documents/Study/Y1S2/DMT/assignment2/feature_engineered_test_chunk_{i//chunk_size}.csv', index=False)

In [51]:
df = pd.read_csv(f'/Users/eva/Documents/Study/Y1S2/DMT/assignment2/feature_engineered_training_chunk_0.csv')

In [52]:
#print number of df columns
print(len(combined_df.columns))


218


In [53]:
# check the NA values columns number
na_cols = combined_df.columns[combined_df.isna().any()].tolist()
print(len(na_cols))

2


In [54]:
# print df columns names
print(df.columns)

Index(['srch_id', 'date_time', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score',
       ...
       'price_diff', 'prop_location_score1_diff', 'prop_location_score2_diff',
       'prop_review_score_diff', 'prop_starrating_diff', 'price_per_adult',
       'price_per_person', 'prop_historical_price', 'prop_clicked_prob',
       'child_bool'],
      dtype='object', length=218)
